Instalamos la librería undetected-chromedriver, que funciona como Selenium pero añade medidas para evitar el ban al hacer WebScraping.

In [2]:
# pip install undetected-chromedriver

Versión que funciona bastante bien. Solo para vehículos de 1 página (65):

In [1]:
import time
from datetime import datetime
import random
import sqlite3
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup as bs
from include.utils.funciones import *

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")
cursor = con.cursor()

# User agents aleatorios:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0",
]

selected_ua = random.choice(USER_AGENTS)
options = uc.ChromeOptions()
options.add_argument(f"user-agent={selected_ua}")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # opcional


# Iniciar navegador
driver = uc.Chrome(options=options)
driver.get("https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=7&ModelIds%5B0%5D=0")
time.sleep(random.uniform(3, 5))

# Aceptar cookies si aparecen
try:
    Cookies = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
    Cookies.click()
    time.sleep(1)
except:
    pass

# Lista de IDs en la BBDD
cursor.execute("SELECT PK_ANUNCIO_ID FROM TX_VEHICULOS_SEG_MANO")
ids_existentes = {fila[0] for fila in cursor.fetchall()}

# Parsear página inicial
soup = bs(driver.page_source, "lxml")
lista_id_vehiculos = [div["data-ad-id"] for div in soup.select("div[data-ad-id]")]

for i in range(1, len(lista_id_vehiculos) + 1):
    id_vehiculo = lista_id_vehiculos[i - 1]

    if id_vehiculo not in ids_existentes:
        try:
            print(f"Procesando vehículo {i}/{len(lista_id_vehiculos)} - ID: {id_vehiculo}")

            # XPath dinámico basado en el índice
            xpath_anuncio = f"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[{i}]/div/div"
            anuncio_vehiculo = driver.find_element(By.XPATH, xpath_anuncio)
            anuncio_vehiculo.click()

            # Esperamos a que cargue la nueva página
            time.sleep(random.uniform(8, 15))

            url_vehiculo = driver.current_url
            soup_vehiculo = bs(driver.page_source, "lxml")

            # Título
            title_element = soup_vehiculo.select_one('h1.mt-TitleBasic-title')
            if not title_element:
                raise Exception("No se encontró el título del vehículo")

            palabras = title_element.get_text(strip=True).split()
            marca_vehiculo = palabras[0]
            modelo_vehiculo = " ".join(palabras[1:])

            # Precio
            precio = None
            string_precio = soup_vehiculo.find("p", class_="mt-CardAdPrice-cashAmount")
            if string_precio:
                texto_precio = string_precio.text.strip().replace('.', '').replace('€', '').strip()
                precio = float(texto_precio)

            # Datos principales
            tabla_datos_vehiculo = soup_vehiculo.select('ul.mt-PanelAdDetails-data li.mt-PanelAdDetails-dataItem')
            dicc = extraer_datos_tecnicos(tabla_datos_vehiculo)

            # Fecha publicación
            hoy = datetime.now()
            fecha_actual = hoy.strftime("%d/%m")
            string_fecha_publicacion = soup_vehiculo.find("p", class_="mt-PanelAdInfo-published")
            fecha_final_publicacion = None
            anyomes_publicacion = None

            if string_fecha_publicacion:
                texto = string_fecha_publicacion.text.strip()
                fecha_str = texto.split(",")[0].replace("Publicado: ", "").strip()[0:5]
                fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")

                anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                anyomes_publicacion = int(f"{anyo}{fecha_publicacion_dt.month:02d}")

            # Insertar en BBDD
            consulta = """
                INSERT OR IGNORE INTO TX_VEHICULOS_SEG_MANO (
                    pk_anuncio_id, marca, modelo, precio, combustible, anyo_vehiculo,
                    kilometraje, potencia, num_puertas, num_plazas, tipo_cambio,
                    tipo_vehiculo, cilindrada_motor, color, provincia, etiqueta_eco,
                    origen_anuncio, fecha_publicacion, anyomes_publicacion, url
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            valores = (
                id_vehiculo,
                marca_vehiculo,
                modelo_vehiculo,
                precio,
                dicc.get("combustible"),
                int(dicc.get("anyo")) if dicc.get("anyo") else None,
                int(dicc.get("kms")) if dicc.get("kms") else None,
                int(dicc.get("potencia")) if dicc.get("potencia") else None,
                int(dicc.get("puertas")) if dicc.get("puertas") else None,
                int(dicc.get("plazas")) if dicc.get("plazas") else None,
                dicc.get("cambio"),
                dicc.get("carroceria"),
                int(dicc.get("cilindrada")) if dicc.get("cilindrada") else None,
                dicc.get("color"),
                dicc.get("ciudad"),
                dicc.get("etiqueta"),
                "WebScraping",
                fecha_final_publicacion,
                anyomes_publicacion,
                url_vehiculo
            )

            cursor.execute(consulta, valores)
            con.commit()

            time.sleep(random.uniform(15, 15))

            driver.back()
            time.sleep(random.uniform(10, 10))

        except Exception as e:
            print(f"❌ Error al procesar ID {id_vehiculo}: {e}")
            try:
                driver.back()
                time.sleep(3)
            except:
                print("⚠️ Fallo al volver atrás, recargando página...")
                driver.get("https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=7&ModelIds%5B0%5D=0")
                time.sleep(random.uniform(3, 5))


KeyboardInterrupt: 

Miramos cuántos coches se han cargado en la BBDD:

In [12]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT * FROM TX_VEHICULOS_SEG_MANO;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# Mostramos los primeros registros (aún no hay):
df

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,56781291,BMW,Serie 3 325xi E90,8290.0,Gasolina,2006,179000,218,4,5,...,Berlina,2497.0,Negro,Barcelona,None,WebScraping,2025-03-25,202503,2025-05-11 17:56:58,https://www.coches.net/bmw-serie-3-325xi-e90-4...
1,57049267,BMW,Serie 1 118d,12000.0,Diesel,2014,200000,143,3,5,...,Berlina,1995.0,Blanco,Murcia,B (amarilla),WebScraping,2025-05-04,202505,2025-05-11 17:50:22,https://www.coches.net/bmw-serie-1-118d-3p-die...
2,57735226,BMW,Serie 1 120d,32900.0,Diesel,2024,25900,190,5,5,...,Berlina,1995.0,Negro,Madrid,C (verde),WebScraping,2025-04-29,202504,2025-05-11 18:04:08,https://www.coches.net/bmw-serie-1-120d-5p-die...
3,59030836,BMW,Serie 3 318d Touring,18890.0,Diesel,2019,114331,150,5,5,...,Familiar,1995.0,Negro,Madrid,C (verde),WebScraping,2025-04-04,202504,2025-05-11 17:37:10,https://www.coches.net/bmw-serie-3-318d-tourin...
4,59371905,BMW,X2 sDrive20i DCT,26900.0,Gasolina,2018,90000,192,5,5,...,SUV,1998.0,Azul,Barcelona,None,WebScraping,2025-05-04,202505,2025-05-11 17:46:43,https://www.coches.net/bmw-x2-sdrive20i-dct-5p...
5,59590083,BMW,Serie 3 320d Auto.,39900.0,Híbrido,2024,22907,190,4,5,...,Berlina,1995.0,Blanco,Toledo,ECO (azul/verde),WebScraping,2025-02-03,202502,2025-05-11 17:47:20,https://www.coches.net/bmw-serie-3-320d-auto.-...
6,59973168,BMW,Serie 1 116d,9990.0,Diesel,2013,142133,116,5,5,...,Berlina,1995.0,Azul,Cantabria,B (amarilla),WebScraping,2025-02-27,202502,2025-05-11 17:46:05,https://www.coches.net/bmw-serie-1-116d-5p-die...
7,60067512,BMW,X3 XDRIVE20D,9500.0,Diesel,2008,217000,177,5,5,...,SUV,1995.0,Blanco,Madrid,B (amarilla),WebScraping,2025-01-15,202501,2025-05-11 17:48:35,https://www.coches.net/bmw-x3-xdrive20d-5p-die...
8,60188159,BMW,Serie 3 320d,17500.0,Diesel,2013,170000,184,4,5,...,Berlina,1995.0,Negro,Murcia,B (amarilla),WebScraping,2025-05-02,202505,2025-05-11 17:59:42,https://www.coches.net/bmw-serie-3-320d-effici...
9,60199438,BMW,Serie 1 116d,14990.0,Diesel,2019,130000,116,5,5,...,Berlina,1496.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-03-27,202503,2025-05-11 17:49:11,https://www.coches.net/bmw-serie-1-116d-5p-die...


Se han cargado 24 coches. Ha intentado scrapear 65 pero en los que hay diferencias en la estructura de algún campo no se han podido cargar.

Vamos a implementar que recorra más de 1 página para que busque más de 65 registros.

In [4]:
import time
from datetime import datetime, timedelta
import random
import sqlite3
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup as bs
from include.utils.funciones import *

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")
cursor = con.cursor()

# User agents aleatorios:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0",
]

options = uc.ChromeOptions()
selected_ua = random.choice(USER_AGENTS)
options.add_argument(f"user-agent={selected_ua}")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # opcional

# Inicializar navegador
driver = uc.Chrome(options=options)


# Marcas de coche scrapeadas:
lista_marcas = ['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT']



pagina = 1
while pagina < 5:

    # Cambiar según la marca o marcas que queramos guardar:

    url_todas_marcas = f'https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=4&MakeIds%5B1%5D=7&MakeIds%5B2%5D=11&MakeIds%5B3%5D=15&MakeIds%5B4%5D=28&MakeIds%5B5%5D=32&MakeIds%5B6%5D=33&MakeIds%5B7%5D=35&MakeIds%5B8%5D=39&MakeIds%5B9%5D=47&ModelIds%5B0%5D=0&ModelIds%5B1%5D=0&ModelIds%5B2%5D=0&ModelIds%5B3%5D=0&ModelIds%5B4%5D=0&ModelIds%5B5%5D=0&ModelIds%5B6%5D=0&ModelIds%5B7%5D=0&ModelIds%5B8%5D=0&ModelIds%5B9%5D=0&Versions%5B0%5D=&Versions%5B1%5D=&Versions%5B2%5D=&Versions%5B3%5D=&Versions%5B4%5D=&Versions%5B5%5D=&Versions%5B6%5D=&Versions%5B7%5D=&Versions%5B8%5D=&Versions%5B9%5D=&pg={pagina}'
    # url_bmw = f"https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=7&ModelIds%5B0%5D=0&pg={pagina}"

    url_pagina = url_todas_marcas
    driver.get(url_pagina)
    time.sleep(random.uniform(4, 6))

    # Aceptar cookies si aparecen
    if pagina == 1:
        try:
            Cookies = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
            Cookies.click()
            time.sleep(1)
        except:
            pass

    # Lista de IDs en la BBDD
    cursor.execute("SELECT PK_ANUNCIO_ID FROM TX_VEHICULOS_SEG_MANO")
    ids_existentes = {fila[0] for fila in cursor.fetchall()}

    soup = bs(driver.page_source, "lxml")
    lista_id_vehiculos = [div["data-ad-id"] for div in soup.select("div[data-ad-id]")]

    if not lista_id_vehiculos:
        print("✅ No hay más anuncios en esta página. Fin del scraping.")
        break

    for i in range(1, len(lista_id_vehiculos) + 1):
        id_vehiculo = lista_id_vehiculos[i - 1]

        if id_vehiculo not in ids_existentes:
            try:
                print(f"Procesando vehículo {i}/{len(lista_id_vehiculos)} - ID: {id_vehiculo}")

                xpath_anuncio = f"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[{i}]/div/div"
                anuncio_vehiculo = driver.find_element(By.XPATH, xpath_anuncio)
                anuncio_vehiculo.click()

                time.sleep(random.uniform(8, 15))

                scroll_y = random.randint(200, 1000)
                driver.execute_script(f"window.scrollBy(0, {scroll_y});")
                time.sleep(random.uniform(1, 2))
                driver.execute_script(f"window.scrollBy(0, {-scroll_y // 2});")
                time.sleep(random.uniform(1, 2))

                try:
                    actions = ActionChains(driver)
                    body = driver.find_element(By.TAG_NAME, "body")
                    actions.move_to_element_with_offset(body, random.randint(0, 300), random.randint(0, 300)).perform()
                    time.sleep(random.uniform(8, 15))
                except Exception as e:
                    print(f"⚠️ No se pudo simular movimiento del ratón: {e}")

                url_vehiculo = driver.current_url
                soup_vehiculo = bs(driver.page_source, "lxml")

                title_element = soup_vehiculo.select_one('h1.mt-TitleBasic-title')
                if not title_element:
                    raise Exception("No se encontró el título del vehículo")

                palabras = title_element.get_text(strip=True).split()
                marca_vehiculo = palabras[0]
                modelo_vehiculo = " ".join(palabras[1:])

                precio = None
                string_precio = soup_vehiculo.find("p", class_="mt-CardAdPrice-cashAmount")
                if string_precio:
                    texto_precio = string_precio.text.strip().replace('.', '').replace('€', '').strip()
                    precio = float(texto_precio)

                tabla_datos_vehiculo = soup_vehiculo.select('ul.mt-PanelAdDetails-data li.mt-PanelAdDetails-dataItem')
                dicc = extraer_datos_tecnicos(tabla_datos_vehiculo)

                hoy = datetime.now()
                fecha_actual = hoy.strftime("%d/%m")
                string_fecha_publicacion = soup_vehiculo.find("p", class_="mt-PanelAdInfo-published")
                fecha_final_publicacion = None
                anyomes_publicacion = None

                # if string_fecha_publicacion:
                #     texto = string_fecha_publicacion.text.strip()
                #     fecha_str = texto.split(",")[0].replace("Publicado: ", "").strip()[0:5]
                #     fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                #     fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")
                #     anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                #     fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                #     anyomes_publicacion = int(f"{anyo}{fecha_publicacion_dt.month:02d}")


                if string_fecha_publicacion:
                    texto = string_fecha_publicacion.text.strip().lower()

                    if "hace" in texto or "hoy" in texto:
                        fecha_final_publicacion = datetime.now().date()
                    elif "ayer" in texto:
                        fecha_final_publicacion = (datetime.now() - timedelta(days=1)).date()
                    else:
                        try:
                            # Ejemplo: "Publicado: 12/03, hace 3 días"
                            fecha_str = texto.split(",")[0].replace("publicado: ", "").strip()[0:5]
                            fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                            fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")

                            # Ajuste de año si es diciembre vs enero
                            anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                            fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                        except Exception as e:
                            print(f"⚠️ No se pudo interpretar la fecha: {texto} - {e}")
                            fecha_final_publicacion = None

                    if fecha_final_publicacion:
                        anyomes_publicacion = int(f"{fecha_final_publicacion.year}{fecha_final_publicacion.month:02d}")
                    else:
                        anyomes_publicacion = None


                consulta = """
                    INSERT OR IGNORE INTO TX_VEHICULOS_SEG_MANO (
                        pk_anuncio_id, marca, modelo, precio, combustible, anyo_vehiculo,
                        kilometraje, potencia, num_puertas, num_plazas, tipo_cambio,
                        tipo_vehiculo, cilindrada_motor, color, provincia, etiqueta_eco,
                        origen_anuncio, fecha_publicacion, anyomes_publicacion, url
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """

                valores = (
                    id_vehiculo,
                    marca_vehiculo,
                    modelo_vehiculo,
                    precio,
                    dicc.get("combustible"),
                    int(dicc.get("anyo")) if dicc.get("anyo") else None,
                    int(dicc.get("kms")) if dicc.get("kms") else None,
                    int(dicc.get("potencia")) if dicc.get("potencia") else None,
                    int(dicc.get("puertas")) if dicc.get("puertas") else None,
                    int(dicc.get("plazas")) if dicc.get("plazas") else None,
                    dicc.get("cambio"),
                    dicc.get("carroceria"),
                    int(dicc.get("cilindrada")) if dicc.get("cilindrada") else None,
                    dicc.get("color"),
                    dicc.get("ciudad"),
                    dicc.get("etiqueta"),
                    "WebScraping",
                    fecha_final_publicacion,
                    anyomes_publicacion,
                    url_vehiculo
                )

                cursor.execute(consulta, valores)
                con.commit()

                time.sleep(random.uniform(15, 15))
                driver.back()
                time.sleep(random.uniform(10, 10))

                if (i % 5 == 0) or (i % 8 == 0):
                    print("⏸️ Pausa larga simulando descanso del usuario...")
                    time.sleep(random.uniform(30, 30))

            except Exception as e:
                print(f"❌ Error al procesar ID {id_vehiculo}: {e}")
                try:
                    driver.back()
                    time.sleep(3)
                except:
                    pass

    pagina += 1

# Cierre final
driver.quit()
con.close()


Procesando vehículo 1/66 - ID: 60486796
Procesando vehículo 2/66 - ID: 59921854
Procesando vehículo 3/66 - ID: 60467215
Procesando vehículo 4/66 - ID: 60525036
❌ Error al procesar ID 60525036: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[4]/div/div"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BBFC53+61635]
	GetHandleVerifier [0x00BBFC94+61700]
	(No symbol) [0x009E05D3]
	(No symbol) [0x00A2899E]
	(No symbol) [0x00A28D3B]
	(No symbol) [0x00A70E12]
	(No symbol) [0x00A4D2E4]
	(No symbol) [0x00A6E61B]
	(No symbol) [0x00A4D096]
	(No symbol) [0x00A1C840]
	(No symbol) [0x00A1D6A4]
	GetHandleVerifier [0x00E44573+2701795]
	GetHandleVerifier [0x00E3FCF6+2683238]
	GetHandleVerifier [0x00E5AA3E+2793134]
	GetHandleV

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x00BBFC53+61635]
	GetHandleVerifier [0x00BBFC94+61700]
	(No symbol) [0x009E05D3]
	(No symbol) [0x009BFBC9]
	(No symbol) [0x00A53CBE]
	(No symbol) [0x00A6DF19]
	(No symbol) [0x00A4D096]
	(No symbol) [0x00A1C840]
	(No symbol) [0x00A1D6A4]
	GetHandleVerifier [0x00E44573+2701795]
	GetHandleVerifier [0x00E3FCF6+2683238]
	GetHandleVerifier [0x00E5AA3E+2793134]
	GetHandleVerifier [0x00BD6915+155013]
	GetHandleVerifier [0x00BDCFFD+181357]
	GetHandleVerifier [0x00BC74A8+92440]
	GetHandleVerifier [0x00BC7650+92864]
	GetHandleVerifier [0x00BB2040+5296]
	BaseThreadInitThunk [0x74CF5D49+25]
	RtlInitializeExceptionChain [0x76F6CF0B+107]
	RtlGetAppContainerNamedObjectPath [0x76F6CE91+561]


In [2]:
import time
from datetime import datetime, timedelta
import random
import sqlite3
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup as bs
from include.utils.funciones import *

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")
cursor = con.cursor()

# User agents aleatorios:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0",
]

options = uc.ChromeOptions()
selected_ua = random.choice(USER_AGENTS)
options.add_argument(f"user-agent={selected_ua}")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # opcional

# Inicializar navegador
driver = uc.Chrome(options=options)


# Marcas de coche scrapeadas:
lista_marcas = ['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT']



pagina = 1
while pagina < 5:

    # Cambiar según la marca o marcas que queramos guardar:

    url_todas_marcas = f'https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=4&MakeIds%5B1%5D=7&MakeIds%5B2%5D=11&MakeIds%5B3%5D=15&MakeIds%5B4%5D=28&MakeIds%5B5%5D=32&MakeIds%5B6%5D=33&MakeIds%5B7%5D=35&MakeIds%5B8%5D=39&MakeIds%5B9%5D=47&ModelIds%5B0%5D=0&ModelIds%5B1%5D=0&ModelIds%5B2%5D=0&ModelIds%5B3%5D=0&ModelIds%5B4%5D=0&ModelIds%5B5%5D=0&ModelIds%5B6%5D=0&ModelIds%5B7%5D=0&ModelIds%5B8%5D=0&ModelIds%5B9%5D=0&Versions%5B0%5D=&Versions%5B1%5D=&Versions%5B2%5D=&Versions%5B3%5D=&Versions%5B4%5D=&Versions%5B5%5D=&Versions%5B6%5D=&Versions%5B7%5D=&Versions%5B8%5D=&Versions%5B9%5D=&pg={pagina}'
    # url_bmw = f"https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=7&ModelIds%5B0%5D=0&pg={pagina}"

    url_pagina = url_todas_marcas
    driver.get(url_pagina)
    time.sleep(random.uniform(4, 6))

    # Aceptar cookies si aparecen
    if pagina == 1:
        try:
            Cookies = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
            Cookies.click()
            time.sleep(1)
        except:
            pass

    # Lista de IDs en la BBDD
    cursor.execute("SELECT PK_ANUNCIO_ID FROM TX_VEHICULOS_SEG_MANO")
    ids_existentes = {fila[0] for fila in cursor.fetchall()}

    soup = bs(driver.page_source, "lxml")
    lista_id_vehiculos = [div["data-ad-id"] for div in soup.select("div[data-ad-id]")]

    if not lista_id_vehiculos:
        print("✅ No hay más anuncios en esta página. Fin del scraping.")
        break

    for i in range(1, len(lista_id_vehiculos) + 1):
        id_vehiculo = lista_id_vehiculos[i - 1]

        if id_vehiculo not in ids_existentes:
            try:
                print(f"Procesando vehículo {i}/{len(lista_id_vehiculos)} - ID: {id_vehiculo}")

                xpath_anuncio = f"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[{i}]/div/div"
                anuncio_vehiculo = driver.find_element(By.XPATH, xpath_anuncio)
                anuncio_vehiculo.click()

                time.sleep(random.uniform(8, 15))

                # ✅ Mantener siempre la pestaña principal activa
                main_window = driver.current_window_handle
                if len(driver.window_handles) > 1:
                    for handle in driver.window_handles:
                        if handle != main_window:
                            driver.switch_to.window(handle)
                            print("🔴 Pestaña externa detectada (posiblemente publicidad). Cerrando...")
                            driver.close()
                    driver.switch_to.window(main_window)

                scroll_y = random.randint(200, 1000)
                driver.execute_script(f"window.scrollBy(0, {scroll_y});")
                time.sleep(random.uniform(1, 2))
                driver.execute_script(f"window.scrollBy(0, {-scroll_y // 2});")
                time.sleep(random.uniform(1, 2))

                try:
                    actions = ActionChains(driver)
                    body = driver.find_element(By.TAG_NAME, "body")
                    actions.move_to_element_with_offset(body, random.randint(0, 300), random.randint(0, 300)).perform()
                    time.sleep(random.uniform(8, 15))
                except Exception as e:
                    print(f"⚠️ No se pudo simular movimiento del ratón: {e}")

                url_vehiculo = driver.current_url
                soup_vehiculo = bs(driver.page_source, "lxml")

                title_element = soup_vehiculo.select_one('h1.mt-TitleBasic-title')
                if not title_element:
                    raise Exception("No se encontró el título del vehículo")

                palabras = title_element.get_text(strip=True).split()
                marca_vehiculo = palabras[0]
                modelo_vehiculo = " ".join(palabras[1:])

                precio = None
                string_precio = soup_vehiculo.find("p", class_="mt-CardAdPrice-cashAmount")
                if string_precio:
                    texto_precio = string_precio.text.strip().replace('.', '').replace('€', '').strip()
                    precio = float(texto_precio)

                tabla_datos_vehiculo = soup_vehiculo.select('ul.mt-PanelAdDetails-data li.mt-PanelAdDetails-dataItem')
                dicc = extraer_datos_tecnicos(tabla_datos_vehiculo)

                hoy = datetime.now()
                fecha_actual = hoy.strftime("%d/%m")
                string_fecha_publicacion = soup_vehiculo.find("p", class_="mt-PanelAdInfo-published")
                fecha_final_publicacion = None
                anyomes_publicacion = None

                # if string_fecha_publicacion:
                #     texto = string_fecha_publicacion.text.strip()
                #     fecha_str = texto.split(",")[0].replace("Publicado: ", "").strip()[0:5]
                #     fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                #     fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")
                #     anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                #     fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                #     anyomes_publicacion = int(f"{anyo}{fecha_publicacion_dt.month:02d}")


                if string_fecha_publicacion:
                    texto = string_fecha_publicacion.text.strip().lower()

                    if "hace" in texto or "hoy" in texto:
                        fecha_final_publicacion = datetime.now().date()
                    elif "ayer" in texto:
                        fecha_final_publicacion = (datetime.now() - timedelta(days=1)).date()
                    else:
                        try:
                            # Ejemplo: "Publicado: 12/03, hace 3 días"
                            fecha_str = texto.split(",")[0].replace("publicado: ", "").strip()[0:5]
                            fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                            fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")

                            # Ajuste de año si es diciembre vs enero
                            anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                            fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                        except Exception as e:
                            print(f"⚠️ No se pudo interpretar la fecha: {texto} - {e}")
                            fecha_final_publicacion = None

                    if fecha_final_publicacion:
                        anyomes_publicacion = int(f"{fecha_final_publicacion.year}{fecha_final_publicacion.month:02d}")
                    else:
                        anyomes_publicacion = None


                consulta = """
                    INSERT OR IGNORE INTO TX_VEHICULOS_SEG_MANO (
                        pk_anuncio_id, marca, modelo, precio, combustible, anyo_vehiculo,
                        kilometraje, potencia, num_puertas, num_plazas, tipo_cambio,
                        tipo_vehiculo, cilindrada_motor, color, provincia, etiqueta_eco,
                        origen_anuncio, fecha_publicacion, anyomes_publicacion, url
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """

                valores = (
                    id_vehiculo,
                    marca_vehiculo,
                    modelo_vehiculo,
                    precio,
                    dicc.get("combustible"),
                    int(dicc.get("anyo")) if dicc.get("anyo") else None,
                    int(dicc.get("kms")) if dicc.get("kms") else None,
                    int(dicc.get("potencia")) if dicc.get("potencia") else None,
                    int(dicc.get("puertas")) if dicc.get("puertas") else None,
                    int(dicc.get("plazas")) if dicc.get("plazas") else None,
                    dicc.get("cambio"),
                    dicc.get("carroceria"),
                    int(dicc.get("cilindrada")) if dicc.get("cilindrada") else None,
                    dicc.get("color"),
                    dicc.get("ciudad"),
                    dicc.get("etiqueta"),
                    "WebScraping",
                    fecha_final_publicacion,
                    anyomes_publicacion,
                    url_vehiculo
                )

                cursor.execute(consulta, valores)
                con.commit()

                time.sleep(random.uniform(15, 15))
                driver.back()
                time.sleep(random.uniform(10, 10))

                if (i % 5 == 0) or (i % 8 == 0):
                    print("⏸️ Pausa larga simulando descanso del usuario...")
                    time.sleep(random.uniform(30, 30))

            except Exception as e:
                print(f"❌ Error al procesar ID {id_vehiculo}: {e}")
                try:
                    driver.back()
                    time.sleep(3)
                except:
                    pass

    pagina += 1

# Cierre final
driver.quit()
con.close()


Procesando vehículo 1/66 - ID: 60488076
Procesando vehículo 2/66 - ID: 60563672
Procesando vehículo 3/66 - ID: 59812406
Procesando vehículo 4/66 - ID: 60332396
❌ Error al procesar ID 60332396: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[4]/div/div"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00C4FC83+61635]
	GetHandleVerifier [0x00C4FCC4+61700]
	(No symbol) [0x00A705D3]
	(No symbol) [0x00AB899E]
	(No symbol) [0x00AB8D3B]
	(No symbol) [0x00B00E12]
	(No symbol) [0x00ADD2E4]
	(No symbol) [0x00AFE61B]
	(No symbol) [0x00ADD096]
	(No symbol) [0x00AAC840]
	(No symbol) [0x00AAD6A4]
	GetHandleVerifier [0x00ED45A3+2701795]
	GetHandleVerifier [0x00ECFD26+2683238]
	GetHandleVerifier [0x00EEAA6E+2793134]
	GetHandle

KeyboardInterrupt: 

### Claude AI:

In [2]:
import time
from datetime import datetime, timedelta
import random
import sqlite3
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup as bs
from include.utils.funciones import *

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")
cursor = con.cursor()

# User agents aleatorios:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0",
]

options = uc.ChromeOptions()
selected_ua = random.choice(USER_AGENTS)
options.add_argument(f"user-agent={selected_ua}")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # opcional

# Inicializar navegador
driver = uc.Chrome(options=options)

# Marcas de coche scrapeadas:
lista_marcas = ['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT']

# Mantener un registro de IDs procesados durante esta ejecución
ids_procesados_sesion = set()

# Lista de IDs en la BBDD
cursor.execute("SELECT PK_ANUNCIO_ID FROM TX_VEHICULOS_SEG_MANO")
ids_existentes = {fila[0] for fila in cursor.fetchall()}

pagina = 1
max_paginas = 25  # Limitar a 5 páginas para pruebas

while pagina < max_paginas:
    print(f"\n🔄 Procesando página {pagina}...")
    
    # URL para todas las marcas configuradas
    url_todas_marcas = f'https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=4&MakeIds%5B1%5D=7&MakeIds%5B2%5D=11&MakeIds%5B3%5D=15&MakeIds%5B4%5D=28&MakeIds%5B5%5D=32&MakeIds%5B6%5D=33&MakeIds%5B7%5D=35&MakeIds%5B8%5D=39&MakeIds%5B9%5D=47&ModelIds%5B0%5D=0&ModelIds%5B1%5D=0&ModelIds%5B2%5D=0&ModelIds%5B3%5D=0&ModelIds%5B4%5D=0&ModelIds%5B5%5D=0&ModelIds%5B6%5D=0&ModelIds%5B7%5D=0&ModelIds%5B8%5D=0&ModelIds%5B9%5D=0&Versions%5B0%5D=&Versions%5B1%5D=&Versions%5B2%5D=&Versions%5B3%5D=&Versions%5B4%5D=&Versions%5B5%5D=&Versions%5B6%5D=&Versions%5B7%5D=&Versions%5B8%5D=&Versions%5B9%5D=&pg={pagina}'
    
    # Cargar la página de listado
    driver.get(url_todas_marcas)
    time.sleep(random.uniform(4, 6))

    # Aceptar cookies si aparecen (solo en la primera página)
    if pagina == 1:
        try:
            cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
            cookies_button.click()
            print("✅ Cookies aceptadas")
            time.sleep(1)
        except Exception as e:
            print(f"ℹ️ No fue necesario aceptar cookies: {e}")

    # Obtener página actualizada después de posibles popups
    soup = bs(driver.page_source, "lxml")
    
    # Extraer IDs de vehículos de la página actual
    vehiculos_elementos = soup.select("div[data-ad-id]")
    lista_id_vehiculos = [div["data-ad-id"] for div in vehiculos_elementos]
    
    if not lista_id_vehiculos:
        print("✅ No hay más anuncios en esta página. Fin del scraping.")
        break
    
    print(f"📋 Encontrados {len(lista_id_vehiculos)} vehículos en la página {pagina}")
    
    # Procesar cada vehículo de la página
    for i, id_vehiculo in enumerate(lista_id_vehiculos, 1):
        # Verificar si ya hemos procesado este ID en esta sesión o existe en BD
        if id_vehiculo in ids_procesados_sesion:
            print(f"⏭️ ID {id_vehiculo} ya procesado en esta sesión, saltando")
            continue
            
        if id_vehiculo in ids_existentes:
            print(f"⏭️ ID {id_vehiculo} ya existe en la base de datos, saltando")
            continue
        
        print(f"\n🚗 Procesando vehículo {i}/{len(lista_id_vehiculos)} - ID: {id_vehiculo}")
        
        # Marcar como procesado antes de comenzar
        ids_procesados_sesion.add(id_vehiculo)
        
        try:
            # Buscar el elemento correcto para hacer clic mediante data-ad-id
            try:
                # Encontrar el elemento por data-ad-id es más fiable que usar XPath fijo
                selector = f"div[data-ad-id='{id_vehiculo}']"
                anuncio_vehiculo = driver.find_element(By.CSS_SELECTOR, selector)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", anuncio_vehiculo)
                time.sleep(random.uniform(1, 2))
                anuncio_vehiculo.click()
            except Exception as e:
                print(f"❌ No se pudo hacer clic en el anuncio: {e}")
                # Volver a cargar la página y continuar con el siguiente vehículo
                driver.get(url_todas_marcas)
                time.sleep(random.uniform(5, 8))
                continue

            # Esperar a que se cargue la página de detalle
            time.sleep(random.uniform(8, 12))

            # Verificar y manejar múltiples pestañas
            main_window = driver.current_window_handle
            if len(driver.window_handles) > 1:
                for handle in driver.window_handles:
                    if handle != main_window:
                        driver.switch_to.window(handle)
                        print("🔴 Pestaña externa detectada. Cerrando...")
                        driver.close()
                driver.switch_to.window(main_window)

             # Simular movimiento de ratón con límites seguros
            try:
                # Obtener dimensiones del viewport (área visible)
                viewport_width = driver.execute_script("return window.innerWidth;")
                viewport_height = driver.execute_script("return window.innerHeight;")
                
                # Calcular coordenadas seguras (80% del viewport para evitar bordes)
                safe_width = int(viewport_width * 0.8)
                safe_height = int(viewport_height * 0.8)
                
                # Mover a una posición aleatoria dentro del área segura
                x_coord = random.randint(100, max(200, safe_width))
                y_coord = random.randint(100, max(200, safe_height))
                
                # Usar JavaScript para mover el mouse (más fiable que ActionChains)
                driver.execute_script(f"""
                    var event = new MouseEvent('mousemove', {{
                        'view': window,
                        'bubbles': true,
                        'cancelable': true,
                        'clientX': {x_coord},
                        'clientY': {y_coord}
                    }});
                    document.dispatchEvent(event);
                """)
                
                print(f"🖱️ Movimiento del ratón simulado a ({x_coord}, {y_coord})")
                time.sleep(random.uniform(1, 3))
            except Exception as e:
                print(f"⚠️ No se pudo simular movimiento del ratón: {e}")
                # Continuar aunque no se pueda simular el movimiento

            # Capturar URL actual y contenido de la página
            url_vehiculo = driver.current_url
            soup_vehiculo = bs(driver.page_source, "lxml")

            # Extraer título y separar marca/modelo
            title_element = soup_vehiculo.select_one('h1.mt-TitleBasic-title')
            if not title_element:
                raise Exception("No se encontró el título del vehículo")

            palabras = title_element.get_text(strip=True).split()
            marca_vehiculo = palabras[0]
            modelo_vehiculo = " ".join(palabras[1:])
            
            print(f"📝 Analizando: {marca_vehiculo} {modelo_vehiculo}")

            # Extraer precio
            precio = None
            string_precio = soup_vehiculo.find("p", class_="mt-CardAdPrice-cashAmount")
            if string_precio:
                texto_precio = string_precio.text.strip().replace('.', '').replace('€', '').strip()
                try:
                    precio = float(texto_precio)
                except ValueError:
                    print(f"⚠️ No se pudo convertir el precio: {texto_precio}")

            # Extraer datos técnicos
            tabla_datos_vehiculo = soup_vehiculo.select('ul.mt-PanelAdDetails-data li.mt-PanelAdDetails-dataItem')
            dicc = extraer_datos_tecnicos(tabla_datos_vehiculo)

            # Procesar fecha de publicación
            hoy = datetime.now()
            fecha_actual = hoy.strftime("%d/%m")
            string_fecha_publicacion = soup_vehiculo.find("p", class_="mt-PanelAdInfo-published")
            fecha_final_publicacion = None
            anyomes_publicacion = None

            if string_fecha_publicacion:
                texto = string_fecha_publicacion.text.strip().lower()

                if "hace" in texto or "hoy" in texto:
                    fecha_final_publicacion = datetime.now().date()
                elif "ayer" in texto:
                    fecha_final_publicacion = (datetime.now() - timedelta(days=1)).date()
                else:
                    try:
                        # Ejemplo: "Publicado: 12/03, hace 3 días"
                        fecha_str = texto.split(",")[0].replace("publicado: ", "").strip()[0:5]
                        fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                        fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")

                        # Ajuste de año si es diciembre vs enero
                        anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                        fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                    except Exception as e:
                        print(f"⚠️ No se pudo interpretar la fecha: {texto} - {e}")
                        fecha_final_publicacion = None

                if fecha_final_publicacion:
                    anyomes_publicacion = int(f"{fecha_final_publicacion.year}{fecha_final_publicacion.month:02d}")

            # Preparar consulta SQL e insertar datos
            consulta = """
                INSERT OR IGNORE INTO TX_VEHICULOS_SEG_MANO (
                    pk_anuncio_id, marca, modelo, precio, combustible, anyo_vehiculo,
                    kilometraje, potencia, num_puertas, num_plazas, tipo_cambio,
                    tipo_vehiculo, cilindrada_motor, color, provincia, etiqueta_eco,
                    origen_anuncio, fecha_publicacion, anyomes_publicacion, url
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            valores = (
                id_vehiculo,
                marca_vehiculo,
                modelo_vehiculo,
                precio,
                dicc.get("combustible"),
                int(dicc.get("anyo")) if dicc.get("anyo") else None,
                int(dicc.get("kms")) if dicc.get("kms") else None,
                int(dicc.get("potencia")) if dicc.get("potencia") else None,
                int(dicc.get("puertas")) if dicc.get("puertas") else None,
                int(dicc.get("plazas")) if dicc.get("plazas") else None,
                dicc.get("cambio"),
                dicc.get("carroceria"),
                int(dicc.get("cilindrada")) if dicc.get("cilindrada") else None,
                dicc.get("color"),
                dicc.get("ciudad"),
                dicc.get("etiqueta"),
                "WebScraping",
                fecha_final_publicacion,
                anyomes_publicacion,
                url_vehiculo
            )

            cursor.execute(consulta, valores)
            con.commit()
            print(f"✅ Insertado vehículo ID {id_vehiculo} en la base de datos")

            # Pausa antes de volver atrás
            time.sleep(random.uniform(3, 5))
            
            # Volver a la página de listado y esperar a que cargue
            driver.back()
            time.sleep(random.uniform(5, 8))
            
            # Verificar que estamos en la página correcta después de volver atrás
            actual_url = driver.current_url
            if str(pagina) not in actual_url or "segunda-mano" not in actual_url:
                print("⚠️ Después de ir hacia atrás no estamos en la página esperada. Recargando...")
                driver.get(url_todas_marcas)
                time.sleep(random.uniform(5, 8))
            
            # Pausa larga cada 5 u 8 vehículos para simular comportamiento humano
            if (i % 5 == 0) or (i % 8 == 0):
                pausa = random.uniform(20, 30)
                print(f"⏸️ Pausa larga de {pausa:.1f} segundos simulando descanso del usuario...")
                time.sleep(pausa)

        except Exception as e:
            print(f"❌ Error al procesar ID {id_vehiculo}: {e}")
            # Intentar volver a la página de listado en caso de error
            try:
                driver.get(url_todas_marcas)
                time.sleep(random.uniform(5, 8))
            except:
                print("⚠️ Error al intentar recargar la página de listado")
                pass

    print(f"✅ Finalizada página {pagina}")
    pagina += 1

# Añadir stats finales
print(f"\n📊 Estadísticas finales:")
print(f"- Páginas procesadas: {pagina-1}")
print(f"- IDs procesados en esta sesión: {len(ids_procesados_sesion)}")

# Cierre final
driver.quit()
con.close()
print("🏁 Proceso finalizado correctamente")


🔄 Procesando página 1...


KeyboardInterrupt: 

## Nueva versión:

In [3]:
import time
from datetime import datetime, timedelta
import random
import sqlite3
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup as bs
from include.utils.funciones import *

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")
cursor = con.cursor()


# Cargar modelos estandarizados
modelos_estandarizados = cargar_modelos_estandarizados(cursor)
print(f"📚 Cargados {sum(len(modelos) for modelos in modelos_estandarizados.values())} modelos estandarizados de {len(modelos_estandarizados)} marcas")

# User agents aleatorios:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0",
]

options = uc.ChromeOptions()
selected_ua = random.choice(USER_AGENTS)
options.add_argument(f"user-agent={selected_ua}")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # opcional

# Inicializar navegador
driver = uc.Chrome(options=options)

# Marcas de coche scrapeadas:
lista_marcas = ['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT']

# Mantener un registro de IDs procesados durante esta ejecución
ids_procesados_sesion = set()

# Lista de IDs en la BBDD
cursor.execute("SELECT PK_ANUNCIO_ID FROM TX_VEHICULOS_SEG_MANO")
ids_existentes = {fila[0] for fila in cursor.fetchall()}

pagina = 1
max_paginas = 25  # Limitar a 5 páginas para pruebas

while pagina < max_paginas:
    print(f"\n🔄 Procesando página {pagina}...")
    
    # URL para todas las marcas configuradas
    url_todas_marcas = f'https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=4&MakeIds%5B1%5D=7&MakeIds%5B2%5D=11&MakeIds%5B3%5D=15&MakeIds%5B4%5D=28&MakeIds%5B5%5D=32&MakeIds%5B6%5D=33&MakeIds%5B7%5D=35&MakeIds%5B8%5D=39&MakeIds%5B9%5D=47&ModelIds%5B0%5D=0&ModelIds%5B1%5D=0&ModelIds%5B2%5D=0&ModelIds%5B3%5D=0&ModelIds%5B4%5D=0&ModelIds%5B5%5D=0&ModelIds%5B6%5D=0&ModelIds%5B7%5D=0&ModelIds%5B8%5D=0&ModelIds%5B9%5D=0&Versions%5B0%5D=&Versions%5B1%5D=&Versions%5B2%5D=&Versions%5B3%5D=&Versions%5B4%5D=&Versions%5B5%5D=&Versions%5B6%5D=&Versions%5B7%5D=&Versions%5B8%5D=&Versions%5B9%5D=&pg={pagina}'
    
    # Cargar la página de listado
    driver.get(url_todas_marcas)
    time.sleep(random.uniform(4, 6))

    # Aceptar cookies si aparecen (solo en la primera página)
    if pagina == 1:
        try:
            cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
            cookies_button.click()
            print("✅ Cookies aceptadas")
            time.sleep(1)
        except Exception as e:
            print(f"ℹ️ No fue necesario aceptar cookies: {e}")

    # Obtener página actualizada después de posibles popups
    soup = bs(driver.page_source, "lxml")
    
    # Extraer IDs de vehículos de la página actual
    vehiculos_elementos = soup.select("div[data-ad-id]")
    lista_id_vehiculos = [div["data-ad-id"] for div in vehiculos_elementos]
    
    if not lista_id_vehiculos:
        print("✅ No hay más anuncios en esta página. Fin del scraping.")
        break
    
    print(f"📋 Encontrados {len(lista_id_vehiculos)} vehículos en la página {pagina}")
    
    # Procesar cada vehículo de la página
    for i, id_vehiculo in enumerate(lista_id_vehiculos, 1):
        # Verificar si ya hemos procesado este ID en esta sesión o existe en BD
        if id_vehiculo in ids_procesados_sesion:
            print(f"⏭️ ID {id_vehiculo} ya procesado en esta sesión, saltando")
            continue
            
        if id_vehiculo in ids_existentes:
            print(f"⏭️ ID {id_vehiculo} ya existe en la base de datos, saltando")
            continue
        
        print(f"\n🚗 Procesando vehículo {i}/{len(lista_id_vehiculos)} - ID: {id_vehiculo}")
        
        # Marcar como procesado antes de comenzar
        ids_procesados_sesion.add(id_vehiculo)
        
        try:
            # Buscar el elemento correcto para hacer clic mediante data-ad-id
            try:
                # Encontrar el elemento por data-ad-id es más fiable que usar XPath fijo
                selector = f"div[data-ad-id='{id_vehiculo}']"
                anuncio_vehiculo = driver.find_element(By.CSS_SELECTOR, selector)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", anuncio_vehiculo)
                time.sleep(random.uniform(1, 2))
                anuncio_vehiculo.click()
            except Exception as e:
                print(f"❌ No se pudo hacer clic en el anuncio: {e}")
                # Volver a cargar la página y continuar con el siguiente vehículo
                driver.get(url_todas_marcas)
                time.sleep(random.uniform(5, 8))
                continue

            # Esperar a que se cargue la página de detalle
            time.sleep(random.uniform(8, 12))

            # Verificar y manejar múltiples pestañas
            main_window = driver.current_window_handle
            if len(driver.window_handles) > 1:
                for handle in driver.window_handles:
                    if handle != main_window:
                        driver.switch_to.window(handle)
                        print("🔴 Pestaña externa detectada. Cerrando...")
                        driver.close()
                driver.switch_to.window(main_window)

             # Simular movimiento de ratón con límites seguros
            try:
                # Obtener dimensiones del viewport (área visible)
                viewport_width = driver.execute_script("return window.innerWidth;")
                viewport_height = driver.execute_script("return window.innerHeight;")
                
                # Calcular coordenadas seguras (80% del viewport para evitar bordes)
                safe_width = int(viewport_width * 0.8)
                safe_height = int(viewport_height * 0.8)
                
                # Mover a una posición aleatoria dentro del área segura
                x_coord = random.randint(100, max(200, safe_width))
                y_coord = random.randint(100, max(200, safe_height))
                
                # Usar JavaScript para mover el mouse (más fiable que ActionChains)
                driver.execute_script(f"""
                    var event = new MouseEvent('mousemove', {{
                        'view': window,
                        'bubbles': true,
                        'cancelable': true,
                        'clientX': {x_coord},
                        'clientY': {y_coord}
                    }});
                    document.dispatchEvent(event);
                """)
                
                print(f"🖱️ Movimiento del ratón simulado a ({x_coord}, {y_coord})")
                time.sleep(random.uniform(1, 3))
            except Exception as e:
                print(f"⚠️ No se pudo simular movimiento del ratón: {e}")
                # Continuar aunque no se pueda simular el movimiento

            # Capturar URL actual y contenido de la página
            url_vehiculo = driver.current_url
            soup_vehiculo = bs(driver.page_source, "lxml")

            # Extraer título y separar marca/modelo
            title_element = soup_vehiculo.select_one('h1.mt-TitleBasic-title')
            if not title_element:
                raise Exception("No se encontró el título del vehículo")

            palabras = title_element.get_text(strip=True).split()
            marca_vehiculo = palabras[0]
            modelo_vehiculo_original = " ".join(palabras[1:])
            
            # Aplicar estandarización del modelo usando el diccionario de referencia
            modelo_vehiculo = encontrar_modelo_estandarizado(
                marca_vehiculo, 
                modelo_vehiculo_original, 
                modelos_estandarizados
            )
            
            if modelo_vehiculo != modelo_vehiculo_original:
                print(f"📝 Modelo estandarizado: {modelo_vehiculo_original} -> {modelo_vehiculo}")
            else:
                print(f"📝 Analizando: {marca_vehiculo} {modelo_vehiculo}")

            # Extraer precio
            precio = None
            string_precio = soup_vehiculo.find("p", class_="mt-CardAdPrice-cashAmount")
            if string_precio:
                texto_precio = string_precio.text.strip().replace('.', '').replace('€', '').strip()
                try:
                    precio = float(texto_precio)
                except ValueError:
                    print(f"⚠️ No se pudo convertir el precio: {texto_precio}")

            # Extraer datos técnicos
            tabla_datos_vehiculo = soup_vehiculo.select('ul.mt-PanelAdDetails-data li.mt-PanelAdDetails-dataItem')
            dicc = extraer_datos_tecnicos(tabla_datos_vehiculo)

            # Procesar fecha de publicación
            hoy = datetime.now()
            fecha_actual = hoy.strftime("%d/%m")
            string_fecha_publicacion = soup_vehiculo.find("p", class_="mt-PanelAdInfo-published")
            fecha_final_publicacion = None
            anyomes_publicacion = None

            if string_fecha_publicacion:
                texto = string_fecha_publicacion.text.strip().lower()

                if "hace" in texto or "hoy" in texto:
                    fecha_final_publicacion = datetime.now().date()
                elif "ayer" in texto:
                    fecha_final_publicacion = (datetime.now() - timedelta(days=1)).date()
                else:
                    try:
                        # Ejemplo: "Publicado: 12/03, hace 3 días"
                        fecha_str = texto.split(",")[0].replace("publicado: ", "").strip()[0:5]
                        fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                        fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")

                        # Ajuste de año si es diciembre vs enero
                        anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                        fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                    except Exception as e:
                        print(f"⚠️ No se pudo interpretar la fecha: {texto} - {e}")
                        fecha_final_publicacion = None

                if fecha_final_publicacion:
                    anyomes_publicacion = int(f"{fecha_final_publicacion.year}{fecha_final_publicacion.month:02d}")

            # Preparar consulta SQL e insertar datos
            consulta = """
                INSERT OR IGNORE INTO TX_VEHICULOS_SEG_MANO (
                    pk_anuncio_id, marca, modelo, precio, combustible, anyo_vehiculo,
                    kilometraje, potencia, num_puertas, num_plazas, tipo_cambio,
                    tipo_vehiculo, cilindrada_motor, color, provincia, etiqueta_eco,
                    origen_anuncio, fecha_publicacion, anyomes_publicacion, url
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            valores = (
                id_vehiculo,
                marca_vehiculo,
                modelo_vehiculo,
                precio,
                dicc.get("combustible"),
                int(dicc.get("anyo")) if dicc.get("anyo") else None,
                int(dicc.get("kms")) if dicc.get("kms") else None,
                int(dicc.get("potencia")) if dicc.get("potencia") else None,
                int(dicc.get("puertas")) if dicc.get("puertas") else None,
                int(dicc.get("plazas")) if dicc.get("plazas") else None,
                dicc.get("cambio"),
                dicc.get("carroceria"),
                int(dicc.get("cilindrada")) if dicc.get("cilindrada") else None,
                dicc.get("color"),
                dicc.get("ciudad"),
                dicc.get("etiqueta"),
                "WebScraping",
                fecha_final_publicacion,
                anyomes_publicacion,
                url_vehiculo
            )

            cursor.execute(consulta, valores)
            con.commit()
            print(f"✅ Insertado vehículo ID {id_vehiculo} en la base de datos")

            # Pausa antes de volver atrás
            time.sleep(random.uniform(3, 5))
            
            # Volver a la página de listado y esperar a que cargue
            driver.back()
            time.sleep(random.uniform(5, 8))
            
            # Verificar que estamos en la página correcta después de volver atrás
            actual_url = driver.current_url
            if str(pagina) not in actual_url or "segunda-mano" not in actual_url:
                print("⚠️ Después de ir hacia atrás no estamos en la página esperada. Recargando...")
                driver.get(url_todas_marcas)
                time.sleep(random.uniform(5, 8))
            
            # Pausa larga cada 5 u 8 vehículos para simular comportamiento humano
            if (i % 5 == 0) or (i % 8 == 0):
                pausa = random.uniform(20, 30)
                print(f"⏸️ Pausa larga de {pausa:.1f} segundos simulando descanso del usuario...")
                time.sleep(pausa)

        except Exception as e:
            print(f"❌ Error al procesar ID {id_vehiculo}: {e}")
            # Intentar volver a la página de listado en caso de error
            try:
                driver.get(url_todas_marcas)
                time.sleep(random.uniform(5, 8))
            except:
                print("⚠️ Error al intentar recargar la página de listado")
                pass

    print(f"✅ Finalizada página {pagina}")
    pagina += 1

# Añadir stats finales
print(f"\n📊 Estadísticas finales:")
print(f"- Páginas procesadas: {pagina-1}")
print(f"- IDs procesados en esta sesión: {len(ids_procesados_sesion)}")

# Cierre final
driver.quit()
con.close()
print("🏁 Proceso finalizado correctamente")

📚 Cargados 384 modelos estandarizados de 10 marcas

🔄 Procesando página 1...
✅ Cookies aceptadas
📋 Encontrados 66 vehículos en la página 1

🚗 Procesando vehículo 1/66 - ID: 60498281
🖱️ Movimiento del ratón simulado a (756, 533)
📝 Modelo estandarizado: X3 xDrive20d xLine -> x3
✅ Insertado vehículo ID 60498281 en la base de datos


KeyboardInterrupt: 

In [4]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT * FROM TX_VEHICULOS_SEG_MANO
--where
--fecha_carga > '2025-05-11 22:33:00'
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# Mostramos los primeros registros (aún no hay):
df

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60498281,BMW,x3,37900.0,Híbrido,2021.0,33200,190.0,5.0,5.0,...,SUV,1995.0,gris,Madrid,ECO (azul/verde),WebScraping,2025-05-07,202505.0,2025-05-18 14:21:01,https://www.coches.net/bmw-x3-xdrive20d-xline-...
1,59710104,PEUGEOT,308,9000.0,Gasolina,2016.0,127300,110.0,5.0,5.0,...,Berlina,1199.0,blanco,Asturias,None,WebScraping,2025-01-25,202501.0,2025-05-18 11:18:51,https://www.coches.net/peugeot-308-5p-allure-1...
2,60479478,RENAULT,clio,11990.0,Diésel,2022.0,123000,100.0,5.0,5.0,...,Berlina,1461.0,blanco,Málaga,None,WebScraping,2025-05-18,202505.0,2025-05-18 10:06:26,https://www.coches.net/renault-clio-equilibre-...
3,60374025,CITROEN,c3,16500.0,Gasolina,2022.0,18150,83.0,5.0,5.0,...,Berlina,1199.0,blanco,Madrid,None,WebScraping,2025-04-19,202504.0,2025-05-18 10:05:00,https://www.coches.net/citroen-c3-puretech-60k...
4,60576644,VOLKSWAGEN,california,67900.0,Diesel,2023.0,12000,150.0,4.0,4.0,...,Monovolumen,1968.0,Blanco,Las Palmas,None,WebScraping,2025-05-18,202505.0,2025-05-18 07:24:21,https://www.coches.net/volkswagen-california-o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36602,35042,SEAT,leon,18590.0,Diésel,2019.0,21153,115.0,5.0,NaN,...,None,NaN,gris,Madrid,None,dataset kaggle,2021-01-14,202101.0,2021-01-15 00:00:00,f6caffb1207bc61d62baa2759cd0a10b
36603,35043,RENAULT,laguna,14500.0,Diésel,2015.0,68750,150.0,2.0,NaN,...,None,NaN,kng,Málaga,None,dataset kaggle,2021-01-13,202101.0,2021-01-15 00:00:00,732b04fb47dd9559dbdcd099c5af484e
36604,35044,PEUGEOT,108,11000.0,Gasolina,2019.0,21422,72.0,5.0,NaN,...,None,NaN,gris,Huelva,None,dataset kaggle,2021-01-08,202101.0,2021-01-15 00:00:00,c10afb53957fb8012707239bdfa66018
36605,35045,AUDI,rs4,47900.0,Gasolina,2013.0,47900,450.0,5.0,NaN,...,None,NaN,gris,Lugo,None,dataset kaggle,2021-01-13,202101.0,2021-01-15 00:00:00,f74363a6688ba11b71b4d980a959b8d3


In [11]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT * FROM TX_VEHICULOS_SEG_MANO
where
--fecha_carga > '2025-05-11 22:33:00'
    pk_anuncio_id in (60568103, 60568101, 60568498)
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# Mostramos los primeros registros (aún no hay):
df

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60568101,FORD,Puma 1.5 Ecoblue Titanium,16900.0,Diesel,2021,83000,120,5,5,...,SUV,1499,Azul,A Coruña,C (verde),WebScraping,2025-05-14,202505,2025-05-15 18:20:47,https://www.coches.net/ford-puma-1.5-ecoblue-8...


In [3]:
df.groupby("marca")["pk_anuncio_id"].count().sort_values(ascending=False)

marca
BMW              106
RENAULT           89
OPEL              81
SEAT              80
CITROEN           76
FORD              59
VOLKSWAGEN        58
PEUGEOT           55
MERCEDES-BENZ     34
AUDI              28
176.885            1
176.910            1
177.052            1
177.946            1
Name: pk_anuncio_id, dtype: int64

In [32]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT * FROM TX_VEHICULOS_SEG_MANO
where
--fecha_carga > '2025-05-13 19:33:00'
marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# # Mostramos los primeros registros (aún no hay):
df

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60543518,176.885,AUDI de segunda mano y ocasión,17990.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-13 19:52:59,https://www.coches.net/segunda-mano/?MakeIds%5...
1,60543182,176.910,AUDI de segunda mano y ocasión,15990.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-13 19:21:08,https://www.coches.net/segunda-mano/?MakeIds%5...
2,60549062,177.052,AUDI de segunda mano y ocasión,7970.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-13 18:45:16,https://www.coches.net/segunda-mano/?MakeIds%5...
3,60445302,177.946,AUDI de segunda mano y ocasión,39900.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-12 19:11:52,https://www.coches.net/segunda-mano/?MakeIds%5...
